# Multimodal Messages with OpenAI Vision

**Note:** This feature is currently in preview and only available on `rc` branches, starting with `0.0.182-rc.0`. The message schema may change in future releases.

OpenAI supports interleaving images with text in input messages with their `gpt-4-vision-preview`. Here's an example of how this looks:

In [1]:
// Deno.env.set("OPENAI_API_KEY", "");

import { ChatOpenAI } from "npm:langchain@0.0.185/chat_models/openai";
import { HumanMessage } from "npm:langchain@0.0.185/schema";

const chat = new ChatOpenAI({
  model: "gpt-4-vision-preview",
  maxTokens: 1024,
});

// Messages can now take an array of content in addition to a string
const hostedImageMessage = new HumanMessage({
  content: [
    {
      type: "text",
      text: "What does this image say?",
    },
    {
      type: "image_url",
      image_url:
        "https://www.freecodecamp.org/news/content/images/2023/05/Screenshot-2023-05-29-at-5.40.38-PM.png",
    },
  ],
});

await chat.invoke([hostedImageMessage]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'The image says "LangChain". It includes a graphic of a parrot on the left, two interlinked rings (or'... 105 more characters,
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain", "schema" ],
  content: 'The image says "LangChain". It includes a graphic of a parrot on the left, two interlinked rings (or'... 105 more characters,
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

You can also construct `ChatPromptTemplate`s with these multimodal messages to create chains:

In [2]:
import { ChatPromptTemplate, MessagesPlaceholder } from "npm:langchain@0.0.185/prompts";
import { StringOutputParser } from "npm:langchain@0.0.185/schema/output_parser";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Answer all questions like a pirate."],
  new MessagesPlaceholder("input"),
]);

const chain = prompt.pipe(chat).pipe(new StringOutputParser());

await chain.invoke({
  input: [
    hostedImageMessage,
  ],
});

`Arrr, matey! The image be showin' the text "LangChain," with a colorful parrot on the port side and `... 52 more characters

You can also pass in a base64 encoded data URL, convenient when reading from fetched images or from the filesystem:

In [3]:
// const imageData = await fs.readFile(path.join(__dirname, "/data/hotdog.jpg"));
// const imageBase64 = imageData.toString("base64");
const imageBase64 = "..."
const chat = new ChatOpenAI({
  model: "gpt-4-vision-preview",
  maxTokens: 1024,
});

const dataUrlMessage = new HumanMessage({
  content: [
    {
      type: "text",
      text: "What does this image contain?",
    },
    {
      type: "image_url",
      image_url: {
        url: `data:image/jpeg;base64,${imageBase64}`,
      },
    },
  ],
});

await chain.invoke({
  input: [
    dataUrlMessage,
  ],
});

"Arrr, matey! What ye be starin' at is a fine image of a hot dog, perched in a soft bun, ready to be "... 182 more characters